# Win Rate Analysis: Bug Investigation

This notebook investigates the discrepancy between `summarize_judged.py` (51.9% win rate) and the correct calculation (93.2% win rate).

**Root Cause**: `summarize_judged.py` uses `winner` (raw A/B position) instead of `winner_model_vs_chosen` (actual semantic winner), ignoring the A/B shuffle.

In [ ]:
import json
from pathlib import Path
from math import sqrt
from collections import Counter

In [ ]:
def wilson_ci(k, n, z=1.96):
    """Wilson score confidence interval for binomial proportion."""
    if n == 0:
        return (0.0, 0.0)
    phat = k / n
    denom = 1 + z**2 / n
    center = (phat + z**2 / (2 * n)) / denom
    margin = z * sqrt((phat * (1 - phat) + z**2 / (4 * n)) / n) / denom
    return center - margin, center + margin


def load_judged_jsonl(path):
    """Load all records from a judged JSONL file."""
    records = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                records.append(json.loads(line))
    return records

## Load Data

In [ ]:
# Path to the judged file from root pipeline
judged_file = Path("../beta=0.1/judged_policy_vs_chosen.jsonl")
records = load_judged_jsonl(judged_file)
print(f"Loaded {len(records)} records from {judged_file.name}")

## 🐛 Buggy Version (what `summarize_judged.py` does)

Uses `winner` field directly, assuming A is always the model.

In [ ]:
def summarize_buggy(records):
    """Buggy version: uses raw 'winner' field (a/b/tie)."""
    win = lose = tie = 0
    
    for ex in records:
        result = ex["winner"]  # BUG: This is raw A/B, not model vs chosen!
        
        if result == "a":
            win += 1   # Assumes A is always model - WRONG!
        elif result == "b":
            lose += 1  # Assumes B is always hh_chosen - WRONG!
        else:
            tie += 1
    
    effective = win + lose
    win_rate = win / effective if effective > 0 else 0.0
    ci_low, ci_high = wilson_ci(win, effective)
    
    return {
        "method": "BUGGY (raw A/B)",
        "win": win,
        "lose": lose,
        "tie": tie,
        "total": len(records),
        "win_rate": win_rate,
        "ci": (ci_low, ci_high)
    }

buggy_result = summarize_buggy(records)
print(f"🐛 BUGGY Result (summarize_judged.py):")
print(f"   Win / Lose / Tie: {buggy_result['win']} / {buggy_result['lose']} / {buggy_result['tie']}")
print(f"   Win-rate: {buggy_result['win_rate']:.1%}")
print(f"   95% CI: [{buggy_result['ci'][0]:.3f}, {buggy_result['ci'][1]:.3f}]")

## ✅ Correct Version (uses `winner_model_vs_chosen`)

Uses the properly mapped field that accounts for A/B shuffling.

In [ ]:
def summarize_correct(records):
    """Correct version: uses 'winner_model_vs_chosen' field."""
    win = lose = tie = 0
    
    for ex in records:
        result = ex["winner_model_vs_chosen"]  # CORRECT: Already mapped!
        
        if result == "model":
            win += 1
        elif result == "chosen":
            lose += 1
        else:
            tie += 1
    
    effective = win + lose
    win_rate = win / effective if effective > 0 else 0.0
    ci_low, ci_high = wilson_ci(win, effective)
    
    return {
        "method": "CORRECT (model vs chosen)",
        "win": win,
        "lose": lose,
        "tie": tie,
        "total": len(records),
        "win_rate": win_rate,
        "ci": (ci_low, ci_high)
    }

correct_result = summarize_correct(records)
print(f"✅ CORRECT Result:")
print(f"   Win / Lose / Tie: {correct_result['win']} / {correct_result['lose']} / {correct_result['tie']}")
print(f"   Win-rate: {correct_result['win_rate']:.1%}")
print(f"   95% CI: [{correct_result['ci'][0]:.3f}, {correct_result['ci'][1]:.3f}]")

## 📊 Why the Bug Happens: A/B Shuffle Analysis

In [ ]:
# Analyze the shuffle distribution
swapped_count = sum(1 for r in records if r.get('is_swapped', False))
not_swapped_count = len(records) - swapped_count

print(f"A/B Position Shuffle Analysis:")
print(f"  Not swapped (A=model, B=chosen): {not_swapped_count} ({not_swapped_count/len(records):.1%})")
print(f"  Swapped (A=chosen, B=model): {swapped_count} ({swapped_count/len(records):.1%})")
print(f"\nWhen swapped=True, 'winner=b' means MODEL wins, but buggy code counts it as LOSS!")

In [ ]:
# Show examples where the bug causes wrong counting
wrong_counts = []
for r in records:
    winner_ab = r['winner']
    winner_mvc = r['winner_model_vs_chosen']
    is_swapped = r.get('is_swapped', False)
    
    # Buggy logic: a=win, b=lose
    buggy_says = 'win' if winner_ab == 'a' else ('lose' if winner_ab == 'b' else 'tie')
    # Correct logic: model=win, chosen=lose
    correct_says = 'win' if winner_mvc == 'model' else ('lose' if winner_mvc == 'chosen' else 'tie')
    
    if buggy_says != correct_says:
        wrong_counts.append({
            'winner_ab': winner_ab,
            'winner_mvc': winner_mvc,
            'is_swapped': is_swapped,
            'buggy_says': buggy_says,
            'correct_says': correct_says
        })

print(f"Misclassified by buggy code: {len(wrong_counts)} / {len(records)} ({len(wrong_counts)/len(records):.1%})")
print(f"\nFirst 5 misclassified examples:")
for w in wrong_counts[:5]:
    print(f"  winner={w['winner_ab']}, is_swapped={w['is_swapped']}, "
          f"actual={w['winner_mvc']} → buggy says {w['buggy_says']}, should be {w['correct_says']}")

## 📈 Comparison Summary

In [ ]:
import pandas as pd

comparison = pd.DataFrame([
    {
        "Method": "Buggy (summarize_judged.py)",
        "Field Used": "winner (a/b)",
        "Wins": buggy_result['win'],
        "Losses": buggy_result['lose'],
        "Ties": buggy_result['tie'],
        "Win Rate": f"{buggy_result['win_rate']:.1%}",
        "95% CI": f"[{buggy_result['ci'][0]:.3f}, {buggy_result['ci'][1]:.3f}]"
    },
    {
        "Method": "Correct",
        "Field Used": "winner_model_vs_chosen",
        "Wins": correct_result['win'],
        "Losses": correct_result['lose'],
        "Ties": correct_result['tie'],
        "Win Rate": f"{correct_result['win_rate']:.1%}",
        "95% CI": f"[{correct_result['ci'][0]:.3f}, {correct_result['ci'][1]:.3f}]"
    }
])

print("\n" + "="*80)
print("COMPARISON SUMMARY")
print("="*80)
comparison

## 🔧 Compare with gpt_judge_hh_final results

In [ ]:
# Load gpt_judge_hh_final summary
gpt_judge_summary_path = Path("../gpt_judge_hh_final/results/summary-beta-0.1.json")

if gpt_judge_summary_path.exists():
    with open(gpt_judge_summary_path) as f:
        gpt_summary = json.load(f)
    
    print("gpt_judge_hh_final Results:")
    print(f"  Total: {gpt_summary['total']}")
    for model, wins in gpt_summary['counts'].items():
        win_rate = gpt_summary['win_rates'][model]
        print(f"  {model}: {wins} ({win_rate:.1%})")
else:
    print(f"File not found: {gpt_judge_summary_path}")

## ✅ Conclusion

The win rate discrepancy was caused by a **bug in `summarize_judged.py`**:

| Issue | Description |
|-------|-------------|
| **Bug** | Uses `winner` (raw A/B) instead of `winner_model_vs_chosen` |
| **Impact** | When A/B is swapped (~50% of cases), wins are counted as losses |
| **Buggy Rate** | ~51.9% |
| **Correct Rate** | ~93.2% |

Both pipelines produce similar **correct** results (~93%) when interpreted properly.